# Import library

In [1]:
!pip install emot tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import json
import re
from emot.emo_unicode import UNICODE_EMOJI

from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

from __future__ import unicode_literals
import operator
import sys

try:
    from html.parser import HTMLParser
except ImportError:
    from HTMLParser import HTMLParser

try:
    import html
except ImportError:
    pass

# Load data

In [3]:
vihsd_path = "/content/drive/MyDrive/LexNorm/KLTN/Data/ViHSD"
vsmec_path = "/content/drive/MyDrive/LexNorm/KLTN/Data/UIT_VSMEC"
visfd_path = "/content/drive/MyDrive/LexNorm/KLTN/Data/UIT_ViSFD"
vihos_path = "/content/drive/MyDrive/LexNorm/KLTN/Data/ViHOS"
vispamreviews = "/content/drive/MyDrive/LexNorm/KLTN/Data/ViSpamReviews"

In [4]:
def split_sentence_by_punctuation(text):
    # Define a regular expression pattern to match punctuation marks
    pattern = r'([.!?{\n}]{1,6})'

    # Use the re.split() function with the pattern
    splits = re.split(pattern, text)

    # Combine adjacent punctuation marks with their preceding split
    # This is done to ensure that each split contains both the punctuation mark and the preceding text
    result = []

    for pair in zip(splits[::2], splits[1::2]):
        result.append("".join(pair))
    if splits[-1]:
        result.append(splits[-1])

    return result

In [5]:
def split_paragraph(text):
    # Define a regular expression pattern to match newline characters
    newline_pattern = r'\n+'

    # Split the text by newline characters
    splits = re.split(newline_pattern, text)

    # Remove empty strings from the splits
    splits = [split for split in splits if split.strip()]

    return splits

In [6]:
def remove_url(text):
    # Define a regular expression pattern to match URLs
    url_pattern = r'https?://\S+|www\.\S+'

    # Find all URLs in the text
    urls = re.findall(url_pattern, text)

    # Split the text by the URLs
    splits = re.split(url_pattern, text)

    # Combine splits and URLs into a single list
    result = []
    for split, url in zip(splits, urls):
        result.append(split)
        #result.append(url)

    # Add the last split (if any) to the result
    if splits[-1]:
        result.append(splits[-1])

    return " ".join(result)

# Tokenize data and create dataset

## ViHSD

In [ ]:
train = pd.read_csv(vihsd_path + '/raw_data/train.csv')
train['type'] = 'train'
dev = pd.read_csv(vihsd_path + '/raw_data/dev.csv')
dev['type'] = 'dev'
test = pd.read_csv(vihsd_path + '/raw_data/test.csv')
test['type'] = 'test'
data = pd.concat([train,dev, test]).reset_index(drop=True)
data['dataset'] = 'ViHSD'
data = data.fillna('None')
data['index'] = data.index
data = data[[ 'dataset', 'type','index', 'free_text', 'label_id']]
data

,dataset,type,index,free_text,label_id
0,ViHSD,train,0,Em được làm fan cứng luôn rồi nè ❤️ reaction q...,0
1,ViHSD,train,1,Đúng là bọn mắt híp lò xo thụt :))) bên việt n...,2
2,ViHSD,train,2,Đậu Văn Cường giờ giống thằng sida hơn à,0
3,ViHSD,train,3,CÔN ĐỒ CỤC SÚC VÔ NHÂN TÍNH ĐỀ NGHI VN. NHÀ NƯ...,2
4,ViHSD,train,4,Từ lý thuyết đến thực hành là cả 1 câu chuyện ...,0
...,...,...,...,...,...
33395,ViHSD,test,33395,Nghe cũng được. Nhưng vẫn cày views,0
33396,ViHSD,test,33396,Pha team up căng cực đến từ vị trí ông chủ đầm...,0
33397,ViHSD,test,33397,Thanh Nhường sao m đọc đc cmt của t thế tiên s...,0
33398,ViHSD,test,33398,"Đúng rồi, nhìn có ria mép , với mặt giống đàn ...",0


In [ ]:
data.to_csv('/content/drive/MyDrive/LexNorm/KLTN/Data/ASTRA/ViHSD.csv', index=False)

## UIT-VSMEC

In [ ]:
train = pd.read_excel(vsmec_path + '/raw_data/train_nor_811.xlsx')
train['type'] = 'train'
dev = pd.read_excel(vsmec_path + '/raw_data/valid_nor_811.xlsx')
dev['type'] = 'dev'
test = pd.read_excel(vsmec_path + '/raw_data/test_nor_811.xlsx')
test['type'] = 'test'
data = pd.concat([train,dev, test]).reset_index(drop=True)
data['dataset'] = 'UIT-VSMEC'
data = data.fillna('None')
data['index'] = data.index
data = data[[ 'dataset', 'type','index', 'Sentence', 'Emotion']]
data

,dataset,type,index,Sentence,Emotion
0,UIT-VSMEC,train,0,cho mình xin bài nhạc tên là gì với ạ,Other
1,UIT-VSMEC,train,1,cho đáng đời con quỷ . về nhà lôi con nhà mày ...,Disgust
2,UIT-VSMEC,train,2,lo học đi . yêu đương lol gì hay lại thích học...,Disgust
3,UIT-VSMEC,train,3,uớc gì sau này về già vẫn có thể như cụ này :)),Enjoyment
4,UIT-VSMEC,train,4,mỗi lần có video của con là cứ coi đi coi lại ...,Enjoyment
...,...,...,...,...,...
6922,UIT-VSMEC,test,6922,lúc thì cuốc đất è cổ ra không có giun mà câu ...,Sadness
6923,UIT-VSMEC,test,6923,các bác dẫn crush qua cây sữa nay mà hứng nước...,Enjoyment
6924,UIT-VSMEC,test,6924,vợ chồng rồi mà trong sáng như này chỉ có tron...,Surprise
6925,UIT-VSMEC,test,6925,bep này vào binh dương dùng xuất,Other


In [ ]:
data.to_csv('/content/drive/MyDrive/LexNorm/KLTN/Data/ASTRA/VSMEC.csv', index=False)

## ViSFD

In [ ]:
train = pd.read_csv(visfd_path + '/raw_data/Train.csv')
train['type'] = 'train'
dev = pd.read_csv(visfd_path + '/raw_data/Dev.csv')
dev['type'] = 'dev'
test = pd.read_csv(visfd_path + '/raw_data/Test.csv')
test['type'] = 'test'
data = pd.concat([train,dev, test]).reset_index(drop=True)
data['dataset'] = 'UIT-ViSFD'
data = data.fillna('None')
data['index'] = data.index
data = data[[ 'dataset', 'type','index', 'comment', 'n_star', 'date_time', 'label']]
data

,dataset,type,index,comment,n_star,date_time,label
0,UIT-ViSFD,train,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,UIT-ViSFD,train,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,UIT-ViSFD,train,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,UIT-ViSFD,train,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,UIT-ViSFD,train,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...
...,...,...,...,...,...,...,...
11117,UIT-ViSFD,test,11117,"Mẫu mã đẹp lung linh. Máy chạy cực nhanh, mượt...",5,13/05/2019,{FEATURES#Positive};{PERFORMANCE#Positive};{DE...
11118,UIT-ViSFD,test,11118,Có ai bị giống mình không máy thì sài bình thư...,4,8/7/2020,{PERFORMANCE#Negative};
11119,UIT-ViSFD,test,11119,Sản phẩm tốt 😊\nAi chơi game cứ mang 1 em về m...,5,31/05/2020,{PERFORMANCE#Positive};{GENERAL#Positive};
11120,UIT-ViSFD,test,11120,Vừa mới mua xong máy rất đẹp nhân viên rất nhi...,5,8/3/2020,{CAMERA#Positive};{BATTERY#Positive};{PERFORMA...


In [ ]:
data.to_csv('/content/drive/MyDrive/LexNorm/KLTN/Data/ASTRA/ViSFD.csv', index=False)

## ViSpamReviews

In [ ]:
train = pd.read_csv(vispamreviews + '/raw_data/train.csv')
train['type'] = 'train'
dev = pd.read_csv(vispamreviews + '/raw_data/dev.csv')
dev['type'] = 'dev'
test = pd.read_csv(vispamreviews + '/raw_data/test.csv')
test['type'] = 'test'
data = pd.concat([train,dev, test]).reset_index(drop=True)
data['dataset'] = 'ViSpamReviews'
data = data.fillna('None')
data['index'] = data.index
data = data[[ 'dataset', 'type','index', 'Comment', 'Label', 'SpamLabel']]
data

,dataset,type,index,Comment,Label,SpamLabel
0,ViSpamReviews,train,0,"Chất lượng sản phẩm tuyệt vời, má",0,0
1,ViSpamReviews,train,1,Jdjd is a waste of my time and place to ask qu...,1,3
2,ViSpamReviews,train,2,Dhnxncbcncncncncnncncncnncncncncncncncnncncncj...,1,3
3,ViSpamReviews,train,3,"giao hàng nhanh, đóng gói cẩn thận, sản phẩm n...",0,0
4,ViSpamReviews,train,4,Bạt mỏng vs lại không có túi đi kèm để cất khi...,0,0
...,...,...,...,...,...,...
19865,ViSpamReviews,test,19865,Sản phẩm OK rất đẹp !cảm ơn Shop Sp rẻ mà rất ...,0,0
19866,ViSpamReviews,test,19866,Rêu được lấy từ rừng về đã rửa sạch đất cát rồ...,1,3
19867,ViSpamReviews,test,19867,Shop giao hàng nhanh nhưng mỗi tội cái shopee ...,1,2
19868,ViSpamReviews,test,19868,Shop ơi em có đặt 2 ram 8g dr4 nhưng máy em nh...,1,3


In [ ]:
data.to_csv('/content/drive/MyDrive/LexNorm/KLTN/Data/ASTRA/ViSpamReviews.csv', index=False)

## ViHOS

In [7]:
vihos_path = "/content/drive/MyDrive/LexNorm/KLTN/Data/ViHOS"
train_BIO = pd.read_csv(vihos_path + '/raw_data/Sequence_labeling_based_version/Syllable/train_BIO_syllable.csv')
train_span = pd.read_csv('/content/drive/MyDrive/LexNorm/KLTN/Data/ViHOS/raw_data/Span_Extraction_based_version/train.csv')
dev_BIO = pd.read_csv(vihos_path + '/raw_data/Sequence_labeling_based_version/Syllable/dev_BIO_syllable.csv')
dev_span = pd.read_csv('/content/drive/MyDrive/LexNorm/KLTN/Data/ViHOS/raw_data/Span_Extraction_based_version/dev.csv')
test_BIO = pd.read_csv(vihos_path + '/raw_data/Sequence_labeling_based_version/Syllable/test_BIO_syllable.csv')
test_span = pd.read_csv('/content/drive/MyDrive/LexNorm/KLTN/Data/ViHOS/raw_data/Test_data/test.csv')

In [8]:
def create_vihos_df(BIO, spans, type):
  df = pd.DataFrame(columns=['dataset', 'type','sentence_id', 'sentence', 'word', 'index_spans', 'tag'])
  sentence_id = 0
  sent_ids =[]
  sentence = []
  tag_lst = []

  for i in range(len(BIO)):
    if (BIO['sentence_id'][i] == sentence_id + 1):
      df.loc[sentence_id] = {
          'dataset': 'ViHOS',
          'type': type,
          'sentence_id': sentence_id,
          'sentence': spans['content'][sentence_id],
          'word': sentence,
          'index_spans': spans['index_spans'][sentence_id],
          'tag': tag_lst
      }
      sentence_id += 1
      sentence = []
      tag_lst = []
    sentence.append(BIO['Word'][i])
    tag_lst.append(BIO['Tag'][i])
    sent_ids.append(sentence_id)

  if sent_ids[-1]:
      df.loc[sentence_id] = {
          'dataset': 'ViHOS',
          'type': type,
          'sentence_id': sentence_id,
          'sentence': spans['content'][sentence_id],
          'word': sentence,
          'index_spans': spans['index_spans'][sentence_id],
          'tag': tag_lst
      }
  return df

In [9]:
df_train = create_vihos_df(train_BIO, train_span, 'train')
df_dev = create_vihos_df(dev_BIO, dev_span, 'dev')
df_test = create_vihos_df(test_BIO, test_span, 'test')

In [10]:
data = pd.concat([df_train,df_dev, df_test]).reset_index(drop=True)
data = data.fillna('None')
data['index'] = data.index
data = data[[ 'dataset', 'type','index', 'sentence', 'word', 'index_spans', 'tag']]
data

,dataset,type,index,sentence,word,index_spans,tag
0,ViHOS,train,0,Dừa lắm :)),"[Dừa, lắm, :, ), )]",[],"[O, O, O, O, O]"
1,ViHOS,train,1,Bấp bênh vl thế,"[Bấp_bênh, vl, thế]","[9, 10]","[O, B-T, O]"
2,ViHOS,train,2,Chắc cũng biết ko tồn tại đc bao lâu nữa nên c...,"[Chắc, cũng, biết, ko, tồn_tại, đc, bao_lâu, n...","[53, 54, 55]","[O, O, O, O, O, O, O, O, O, O, B-T]"
3,ViHOS,train,3,Thấy chán ad page này kiến thức thì nông cản c...,"[Thấy, chán, ad, page, này, kiến_thức, thì, nô...","[5, 6, 7, 8, 36, 37, 38, 39, 40, 41, 42, 43, 6...","[O, B-T, O, O, O, O, O, B-T, I-T, O, O, O, O, ..."
4,ViHOS,train,4,Giang Giang Đỗ Thị Ngọc Hà trend mới kìa kìa,"[Giang_Giang_Đỗ_Thị, Ngọc_Hà, trend, mới, kìa,...",[],"[O, O, O, O, O, O]"
...,...,...,...,...,...,...,...
11051,ViHOS,test,11051,từ dịch này thấy ý thức người nhật xăng đổ nửa...,"[từ_dịch, này, thấy, ý_thức, người, nhật, xăng...","[137, 138, 139, 140, 141, 142, 143, 144, 145, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
11052,ViHOS,test,11052,Đang nói chuyện này cái ổng bẻ lái qua chuyện ...,"[Đang, nói_chuyện, này, cái, ổng, bẻ_lái, qua,...",[],"[O, O, O, O, O, O, O, O, O, O, O, O]"
11053,ViHOS,test,11053,Ôi trời giai cấp lđ ở ta thì đổi trắng thay đe...,"[Ôi, trời, giai_cấp, lđ, ở, ta, thì, đổi_trắng...","[29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4...","[O, O, O, O, O, O, O, B-T, O, O, O, O, O, O, O..."
11054,ViHOS,test,11054,Đéo ai làm thế,"[Đéo, ai, làm, thế]","[0, 1, 2]","[B-T, O, O, O]"


In [ ]:
data.to_csv('/content/drive/MyDrive/LexNorm/KLTN/Data/ASTRA/ViHOS.csv', index=False)